<a href="https://colab.research.google.com/github/WorldModelG15/final-task/blob/takata/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recursive -b takata https://github.com/WorldModelG15/final-task.git
%cd final-task

fatal: destination path 'final-task' already exists and is not an empty directory.
/content/final-task


In [2]:
!pip install pyvirtualdisplay

!apt-get install python-opengl -y
!apt install xvfb -y
!apt-get install x11-utils -y
from pyvirtualdisplay import Display
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
def create_display():
    display = Display(visible=0, size=(1400, 900))
    display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !sh xvfb start
        %env DISPLAY=:1
create_display()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [3]:
%cd gym-duckietown/
!pip install -e .
%cd ..

/content/final-task/gym-duckietown
Obtaining file:///content/final-task/gym-duckietown
  Attempting uninstall: duckietown-gym-daffy
    Found existing installation: duckietown-gym-daffy 6.1.31
    Can't uninstall 'duckietown-gym-daffy'. No files were found to uninstall.
  Running setup.py develop for duckietown-gym-daffy
/content/final-task


In [4]:
%cd final-task

[Errno 2] No such file or directory: 'final-task'
/content/final-task


In [4]:
import sys
sys.path.append('/content/final-task/gym-duckietown/src/')
from utils.env import launch_env
from utils.wrappers import (
    EarlyStopWrapper,
    NormalizeWrapper,
    ImgWrapper,
    DtRewardWrapper,
    ActionWrapper,
    ResizeWrapper,
    EarlyStopWrapper,
)
from dreamer.config import DreamerConfig
from dreamer.trainer import Trainer
import torch

env = launch_env(map_name="loop_pedestrians")
env = ResizeWrapper(env)
# env = NormalizeWrapper(env)
env = ImgWrapper(env)  # to make the images from 120x160x3 into 3x120x160
env = ActionWrapper(env)
env = DtRewardWrapper(env)
env = EarlyStopWrapper(env)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
config = DreamerConfig()
trainer = Trainer(env, device, config)

trainer.train()


DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.2.38 path /usr/local/lib/python3.7/dist-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.7/dist-packages
DEBUG:aido_schemas:aido-protocols version 6.0.59 path /usr/local/lib/python3.7/dist-packages
DEBUG:nodes:version 6.2.13 path /usr/local/lib/python3.7/dist-packages pyparsing 3.0.7


{'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


INFO:duckietown_world: data: /usr/local/lib/python3.7/dist-packages/duckietown_world/data


episode [   6/ 300] is collected. Total reward is -991.560542
elasped time for interaction: 17.25s
update_step:   1 model loss: 3570.00244, kl_loss: 3.00000, obs_loss: 3566.97559, reward_loss: 0.02677, value_loss: 0.06055 action_loss: 0.22380
update_step:   2 model loss: 3447.91089, kl_loss: 3.00000, obs_loss: 3444.87769, reward_loss: 0.03315, value_loss: 0.03815 action_loss: 0.17606
update_step:   3 model loss: 3425.03223, kl_loss: 3.00000, obs_loss: 3422.01465, reward_loss: 0.01765, value_loss: 0.02252 action_loss: 0.13506
update_step:   4 model loss: 3496.52441, kl_loss: 3.00000, obs_loss: 3493.48608, reward_loss: 0.03841, value_loss: 0.01108 action_loss: 0.09041
update_step:   5 model loss: 3439.51538, kl_loss: 3.00000, obs_loss: 3436.48975, reward_loss: 0.02556, value_loss: 0.00469 action_loss: 0.05060
update_step:   6 model loss: 3544.37476, kl_loss: 3.00000, obs_loss: 3541.33350, reward_loss: 0.04129, value_loss: 0.00245 action_loss: 0.00453
update_step:   7 model loss: 3495.646

KeyboardInterrupt: ignored